In [ ]:
import os.path
import warnings
import fiona
import geopandas
import rasterio
import rasterio.plot
from rasterio.windows import Window
import folium
import matplotlib.pyplot as plt

In [ ]:
GDB = '/data/projects/kreike/data/KreikeSampleExtractedDataNam52022.gdb/'
TIFF = '/data/projects/kreike/data/Aerial1970_May12_2021_secondGCPsTest.tif'

In [ ]:
def subset(input_tif, output_tif, gdf, band_index=1):
    with rasterio.open(input_tif) as raster:
        if raster.crs != gdf.crs:
            raise AssertionError

        _minx, _miny, _maxx, _maxy = tuple(gdf.total_bounds)
        _col_off1, _row_off1 = ~raster.transform * (_minx, _miny)
        _col_off2, _row_off2 = ~raster.transform * (_maxx, _maxy)
        _width, _height = _col_off2-_col_off1, _row_off1-_row_off2

        window = Window(
            _col_off1,
            _row_off2,
            _width,
            _height
        )

        band = raster.read(
            band_index,
            window=window
        )

    with rasterio.open(
            output_tif,
            mode='w',
            driver='GTiff',
            height=band.shape[0],
            width=band.shape[1],
            count=1,
            dtype=band.dtype,
            crs=raster.crs.wkt,
            transform=rasterio.windows.transform(window, raster.transform),
    ) as new_dataset:
        new_dataset.write(band, indexes=1)

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
# Set index to the country name (unique)
world = world.set_index('name', drop=False)

africa = world[(world['continent'] == 'Africa')]
# Some attributes, like area/centroid/boundary, are available to us for 'world', since it has a geometry column
print(africa.area['Angola'])  # world.area is a Series, indexed by the same index as the DataFrame (country)

# The active geometry of a GeoDataFrame gives us a GeoSeries that we can plot/explore
# africa.plot()  # explore() works in jupyter
roi = africa[((africa['name'] == 'Angola') | (africa['name'] == 'Namibia'))]
roi = africa

layers = fiona.listlayers(GDB)
print(layers)
gdf = geopandas.read_file(GDB, layer='Omuti1972')

roi = roi.to_crs(gdf.crs)
fig, ax = plt.subplots(nrows=1, ncols=1, sharey='all', sharex='all')
ax.ticklabel_format(useOffset=False, style='plain')

subset(TIFF, 'subset.tif', gdf)
with rasterio.open('subset.tif', 'r') as raster:
    rasterio.plot.show(raster, with_bounds=True, ax=ax)
    roi.plot(ax=ax, color='lightgrey', edgecolor=None)
    gdf.plot(ax=ax, color='blue')

In [ ]:
gdf.explore()

In [ ]:
raster_bounds = None
with rasterio.open('../subset_reprojected.png') as raster:
    b = raster.bounds  # left, bottom, right, top
    raster_bounds = [[b[1], b[0]], [b[3], b[2]]]  # [[lat_min, lon_min], [lat_max, lon_max]]
        
geo_json1 = africa.geometry.to_json()
# Folium by default accepts lat/long (crs 4326) as input
geo_json2 = gdf.to_crs(epsg=4326).geometry.to_json()
geo_j1 = folium.GeoJson(data=geo_json1)
geo_j2 = folium.GeoJson(data=geo_json2)

map = folium.Map()
geo_j1.add_to(map)
geo_j2.add_to(map)

overlay = folium.raster_layers.ImageOverlay(
    name="Aerial Image",
    image='../subset_reprojected.png',
    bounds=raster_bounds,
    interactive=True,
    cross_origin=True,
    zindex=0,
)
overlay.add_to(map)
folium.LayerControl().add_to(map)
    
map

In [ ]:
gdf = geopandas.read_file(GDB, layer='Omuti1972')
gdf.explore()